In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import parallel_where_binary, Aligner4
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

In [ ]:
stack_fixed = 'MD590'

In [ ]:
train_sample_scheme = 1
global_transform_scheme = 1

local_transform_scheme = 3
# 1: no regularization
# 2: with regularization
# 3: no regularization, with surround
# 4: with regularization, with surround

if local_transform_scheme == 1:
    reg_weights = np.array([0.,0.,0.])
elif local_transform_scheme == 2:
    reg_weights = np.array([1e-4, 1e-4, 1e-4])
elif local_transform_scheme == 3:
    reg_weights = np.array([0.,0.,0.])
elif local_transform_scheme == 4:
    reg_weights = np.array([1e-4, 1e-4, 1e-4])
    
stack_moving = 'atlas_on_MD589'

In [ ]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

In [ ]:
label_to_name_fixed = {i+1: name for i, name in enumerate(sorted(structures))}
name_to_label_fixed = {n:l for l, n in label_to_name_fixed.iteritems()}

In [ ]:
structures_sided = sum([[n] if n in singular_structures 
                        else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

In [ ]:
if local_transform_scheme == 1 or local_transform_scheme == 2:
    
    label_to_name_moving = {i+1: name for i, name in enumerate(structures_sided)}
    name_to_label_moving = {n:l for l, n in label_to_name_moving.iteritems()}

elif local_transform_scheme == 3 or local_transform_scheme == 4:

    structures_sided_plus_surround = sum([[s, s+'_surround'] for s in structures_sided], [])

    label_to_name_moving = {i+1: name for i, name in enumerate(structures_sided_plus_surround)}
    name_to_label_moving = {n:l for l, n in label_to_name_moving.iteritems()}

In [ ]:
def convert_to_original_name(name):
    return name.split('_')[0]

labelIndexMap_m2f = {}
for label_m, name_m in label_to_name_moving.iteritems():
    labelIndexMap_m2f[label_m] = name_to_label_fixed[convert_to_original_name(name_m)]

In [ ]:
label_weights_m = {}
for label_m, name_m in label_to_name_moving.iteritems():
    if 'surround' in name_m:
        if local_transform_scheme == 1 or local_transform_scheme == 2:
            label_weights_m[label_m] = 0
        elif local_transform_scheme == 3 or local_transform_scheme == 4:
            label_weights_m[label_m] = -1
    else:
        label_weights_m[label_m] = 1

In [ ]:
volume_fixed = {name_to_label_fixed[name]: DataManager.load_score_volume(stack=stack_fixed, label=name, downscale=32, train_sample_scheme=train_sample_scheme)
               for name in structures}

print volume_fixed.values()[0].shape
print volume_fixed.values()[0].dtype

vol_fixed_xmin, vol_fixed_ymin, vol_fixed_zmin = (0,0,0)
vol_fixed_ymax, vol_fixed_xmax, vol_fixed_zmax = np.array(volume_fixed.values()[0].shape) - 1
vol_fixed_xdim = vol_fixed_xmax + 1 - vol_fixed_xmin
vol_fixed_ydim = vol_fixed_ymax + 1 - vol_fixed_ymin
vol_fixed_zdim = vol_fixed_zmax + 1 - vol_fixed_zmin

In [ ]:
if local_transform_scheme == 1 or local_transform_scheme == 2:

    volume_moving = {name_to_label_moving[name_s]: DataManager.load_transformed_volume(stack_m='atlas_on_MD589',
                                                                                       type_m='score',
                                                                                       stack_f=stack_fixed,
                                                                                       type_f='score',
                                                                                       downscale=32,
                                                                                       train_sample_scheme_f=train_sample_scheme,
                                                                                       label=name_s)
                     for name_s in structures_sided}

elif local_transform_scheme == 3 or local_transform_scheme == 4:
    
    volume_moving = {name_to_label_moving[name_s]: DataManager.load_transformed_volume(stack_m='atlas_on_MD589',
                                                                                       type_m='score',
                                                                                       stack_f=stack_fixed,
                                                                                       type_f='score',
                                                                                       downscale=32,
                                                                                       train_sample_scheme_f=train_sample_scheme,
                                                                                       label=name_s)
                     for name_s in structures_sided_plus_surround}
    

print volume_moving.values()[0].shape
print volume_moving.values()[0].dtype

vol_moving_xmin, vol_moving_ymin, vol_moving_zmin = (0,0,0)
vol_moving_ymax, vol_moving_xmax, vol_moving_zmax = np.array(volume_moving.values()[0].shape) - 1

In [1]:
import numdifftools as nd
# https://media.readthedocs.org/pdf/numdifftools/latest/numdifftools.pdf

In [ ]:
# for name_s in structures_sided:
for name_s in ['IC']:
    
    print name_s

    if local_transform_scheme == 1 or local_transform_scheme == 2:
    
        aligner = Aligner4(volume_fixed, {name_to_label_moving[name_s]: 
                                          volume_moving[name_to_label_moving[name_s]]}, \
                           labelIndexMap_m2f={name_to_label_moving[name_s]:
                                              name_to_label_fixed[convert_name_to_unsided(name_s)]})

    elif local_transform_scheme == 3 or local_transform_scheme == 4:
                
        aligner = Aligner4(volume_fixed, {name_to_label_moving[name_s]: volume_moving[name_to_label_moving[name_s]],
                                         name_to_label_moving[name_s+'_surround']: volume_moving[name_to_label_moving[name_s+'_surround']]}, \
                        labelIndexMap_m2f={name_to_label_moving[name_s]: name_to_label_fixed[convert_name_to_unsided(name_s)],
                                          name_to_label_moving[name_s+'_surround']: name_to_label_fixed[convert_to_original_name(name_s+'_surround')]})

    # aligner.set_centroid(centroid_m='volume_centroid', centroid_f='volume_centroid')
    aligner.set_centroid(centroid_m='structure_centroid', centroid_f='centroid_m', 
                         indices_m=[name_to_label_moving[name_s]])

In [ ]:
def perturb(tx, ty, tz):
    return aligner.compute_score(T + [0,0,0,tx,0,0,0,ty,0,0,0,tz])

In [ ]:
dets = []
step_sizes = np.linspace(1, 20, 11)

for step_size in step_sizes:
    h = nd.Hessian(lambda (tx, ty, tz): perturb(tx, ty, tz), step=(step_size, step_size, step_size))
    H = h((0,0,0))
#     s, v = np.linalg.eigh(H)
    d = np.abs(np.linalg.det(H))
    dets.append(d)

In [ ]:
plt.plot(dets);
plt.xlabel('step size');
plt.xticks(range(len(step_sizes)), map(lambda x: '%.2f' % x, step_sizes));
plt.ylabel('determinant of Hessian matrix');